# 2. Overfitting 

En esta sección, vamos a simular un proceso de generación de datos (PGD) para ilustrar el concepto de **overfitting** (sobreajuste).

In [ ]:
# !pip install numpy scikit-learn seaborn matplotlib
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LassoCV, Lasso
from sklearn.metrics import r2_score
import random

# 2. Definimos los parámetros de la simulación

In [ ]:
#parametros
n = 1000
features = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

#generacion de datos, PGD
np.random.seed(0)
x = np.random.uniform(0, 1, n)
epsilon_y = np.random.normal(0, 1, n)
x.sort()
x = x.reshape(-1, 1)
# print(x) 

Ahora creamos y sin intercepto